# 顔の検出と分析

コンピュータビジョンソリューションは、人間の顔を検出、分析、または識別するために人工知能（AI）ソリューションを必要とすることがよくあります。例えば、小売企業のノースウインド・トレーダーズが「スマートストア」の導入を決めたとします。これを実現するための1つの方法として、顔の検出と分析、つまり画像に顔が写っているかどうかを判断し、写っている場合はその特徴を分析することが挙げられます。


<p style='text-align:center'><img src='./images/face_analysis.jpg' alt='顔を分析するロボット'/></p>


## 顔認識サービスを使って顔を検出する

Northwind Tradersが作りたいスマートストアシステムが、顧客を検知して顔の特徴を分析できる必要があるとします。Microsoft Azureでは、**Face**コグニティブサービスを使用してこれを行うことができます。

まずはAzureサブスクリプションで**Cognitive Services**リソースを作成してみましょう。

> **Note**: すでにCognitive Servicesリソースがある場合は、Azureポータルの**クイックスタート**ページを開き、そのキーとエンドポイントを以下のセルにコピーします。それ以外の場合は、以下の手順に従ってリソースを作成します。

1. 別のブラウザタブで、https://portal.azure.com の Azure ポータルを開き、Microsoft アカウントでサインインします。

2. **&#65291;Create a resource** ボタンをクリックして、*コグニティブサービス**を検索し、次の設定で**コグニティブサービス**リソースを作成します:
    - **Name**: *一意の名前を入力してください*
    - **Subscription**: *あなたのAzureサブスクリプション*
    - **Location**: *利用可能なリージョンを選択:
    - **Pricing tier**: S0
    - **Resource group**: *固有の名前を持つリソースグループを作成します。*
3. デプロイが完了するのを待ちます。次に、コグニティブサービスリソースに移動し、**クイックスタート**ページで、キーとエンドポイントに注意してください。クライアント アプリケーションからコグニティブ サービス リソースに接続するには、これらが必要です。

4. リソースの **Key1** をコピーして、**YOUR_COG_KEY** を置き換えて、以下のコードに貼り付けます。
5. リソースの **endpoint** をコピーして、以下のコードに貼り付け、 **YOUR_COG_ENDPOINT** を置き換えます。
6. 下のセルの緑色の<span style="color:green">&#9655;</span>ボタン（セルの左上）をクリックして、下のセルでコードを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

これでCognitive Servicesのセットアップが完了しました。

そしてPythonから本演習を実行するには、まずAzureの関連パッケージをインストールする必要があります。

In [ ]:
! pip install azure-cognitiveservices-vision-face

これで準備が完了しまして、Faceサービスを使って店内の人の顔を検出することができます。

以下のコードセルを実行して例を見てみましょう。

In [ ]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import faces
import os
%matplotlib inline

# 顔検出クライアントを作成します.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# 画像を開く
image_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_stream = open(image_path, "rb")

# 顔を検出
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# 顔を表示する (コードは python_code/faces.py にあります)
faces.show_faces(image_path, detected_faces)

検出された各顔には固有のIDが割り当てられているので、アプリケーションは検出された個々の顔を識別することができます。

以下のセルを実行して、さらにいくつかの買い物客の顔のIDを確認してください。

In [ ]:
# 画像を開く
image_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_stream = open(image_path, "rb")

# 顔を検出
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# 顔を表示する (コードは python_code/faces.py にあります)
faces.show_faces(image_path, detected_faces, show_id=True)

## 顔の属性を分析する

Face cognitive サービスは、単に顔を検出するだけでなく、顔の特徴や表情を分析し、性別、年齢、感情状態を示唆することができます。また、顔の特徴や表情を分析して、性別、年齢、感情の状態を示唆することもできます。

In [ ]:
# 画像を開く
image_path = os.path.join('data', 'face', 'store_cam1.jpg')
image_stream = open(image_path, "rb")

# 顔と指定された顔の属性を検出します．
attributes = ['age', 'gender', 'emotion']
detected_faces = face_client.face.detect_with_stream(image=image_stream, return_face_attributes=attributes)

# 顔と属性を表示する (コードは python_code/faces.py にあります)
faces.show_face_attributes(image_path, detected_faces)

画像に写っているお客様の感情スコアに基づいて、彼女は買い物に満足しているように見えます。

## 似たような顔を探す

検出された顔ごとに作成される顔ＩＤは、検出された顔を個別に識別するために使用されます。これらのIDを使用して、検出された顔を以前に検出された顔と比較したり、類似した特徴を持つ顔を見つけたりすることができます。

例えば、以下のセルを実行して、ある画像の買い物客と別の画像の買い物客を比較し、一致する顔を見つけます。

In [ ]:
# 画像1の最初の顔のIDを取得します．
image_1_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_1_stream = open(image_1_path, "rb")
image_1_faces = face_client.face.detect_with_stream(image=image_1_stream)
face_1 = image_1_faces[0]

# 2枚目の画像で顔のIDを取得します
image_2_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_2_stream = open(image_2_path, "rb")
image_2_faces = face_client.face.detect_with_stream(image=image_2_stream)
image_2_face_ids = list(map(lambda face: face.face_id, image_2_faces))

# 画像2から画像1の顔と似ている顔を探す
similar_faces = face_client.face.find_similar(face_id=face_1.face_id, face_ids=image_2_face_ids)

# 画像1に顔を表示し，画像2に類似した顔を表示します(code in python_code/face.py)

faces.show_similar_faces(image_1_path, face_1, image_2_path, image_2_faces, similar_faces)

## 顔を認識する

ここまでで、Faceサービスが顔と顔の特徴を検出し、似たような2つの顔を識別できることを見てきました。顔認識*ソリューションを実装して、特定の人の顔を認識するようにFaceサービスのリソースを訓練することで、さらに一歩前進することができます。これは、ソーシャルメディアアプリケーションで友人の写真を自動的にタグ付けしたり、バイオメトリクス認証システムの一部として顔認証を使用したりするなど、さまざまなシナリオで役立ちます。

これがどのように機能するかを確認するために、ノースウインド・トレーダーズ社が顔認証を使用して、IT部門の権限のある従業員だけが安全なシステムにアクセスできるようにしたいと考えているとします。

権限を与えられた従業員を代表する*person group*を作成することから始めます。

In [ ]:
group_id = 'employee_group_id'
try:
    # 既に存在する場合はグループを削除します
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')

これで*personグループ*ができたので、グループに入れたい社員ごとに*person*を追加し、顔写真を複数枚登録して、フェイスサービスが顔の特徴を知ることができるようになります。理想的には、同じ人が異なるポーズや表情で写っているものが良いでしょう。

ウェンデルという一人の社員を追加し、彼の写真を3枚登録します。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# 人(Wendell)をグループに追加する
wendell = face_client.person_group_person.create(group_id, 'Wendell')

# Wendellの写真を手に入れる
folder = os.path.join('data', 'face', 'wendell')
wendell_pics = os.listdir(folder)

# 写真を登録する
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in wendell_pics:
    # グループ内の人物に写真を追加する
    img_path = os.path.join(folder, pic)
    img_stream = open(img_path, "rb")
    face_client.person_group_person.add_face_from_stream(group_id, wendell.person_id, img_stream)

    #  各画像を表示する
    img = Image.open(img_path)
    i +=1
    a=fig.add_subplot(1,len(wendell_pics), i)
    a.axis('off')
    imgplot = plt.imshow(img)
plt.show()

人物が追加され、写真が登録されたことで、一人一人を認識するためのFaceサービスのトレーニングができるようになりました。

In [ ]:
face_client.person_group.train(group_id)
print('Trained!')

これでモデルが訓練されたので、画像中の認識された顔を識別するのに使用することができます。

In [ ]:
# 2枚目の画像で顔のIDを取得します
image_path = os.path.join('data', 'face', 'employees.jpg')
image_stream = open(image_path, "rb")
image_faces = face_client.face.detect_with_stream(image=image_stream)
image_face_ids = list(map(lambda face: face.face_id, image_faces))

# 認識された顔の名前を取得する
face_names = {}
recognized_faces = face_client.face.identify(image_face_ids, group_id)
for face in recognized_faces:
    person_name = face_client.person_group_person.get(group_id, face.candidates[0].person_id).name
    face_names[face.face_id] = person_name

# 認識した顔を表示
faces.show_recognized_faces(image_path, image_faces, face_names)



## Learn More

Cognitive ServicesのFaceについては、[Faceドキュメント](https://docs.microsoft.com/azure/cognitive-services/face/)を参照してください。
